# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [10]:
# 1) Seu código aqui

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

r = requests.get(url)

data_json = r.json()

df = pd.DataFrame(data_json['items'])

In [12]:
# 2) Seu código aqui
status_freq = df['status'].value_counts().head(3)
print("Três status das solicitações mais frequentes:")
print(status_freq)

Três status das solicitações mais frequentes:
Deferido        96
Arquivado       53
Regularizado    26
Name: status, dtype: int64


In [17]:
# 3) Seu código aqui
df['ano_status'] = df['data_status'].str.extract(r'(\d{4})')

ano_freq = df['ano_status'].value_counts()
print("Frequência de cada ano do campo 'ano_status':")
print(ano_freq)

Frequência de cada ano do campo 'ano_status':
2008    37
2007    29
2014    20
2010    19
2023    17
2019    17
2020    15
2013    15
2012    13
2022    11
2011    10
2009     9
2015     8
2021     5
2017     5
2006     4
2018     4
2002     4
2016     4
2003     1
2004     1
Name: ano_status, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [35]:
#1) Seu código aqui
def consultar_dados_uf_tipo(uf, tipo_interessado):
    url = f"https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}"
    r = requests.get(url)
    data_json = r.json()
    df = pd.DataFrame(data_json['items'])
    return df

In [44]:
# 2) Seu código aqui
df_mg = consultar_dados_uf_tipo('MG', 'Estado')
solicitacoes_mg_arquivadas = df_mg[df_mg['status'] == 'Arquivado por decurso de prazo']
total_solicitacoes_mg_arquivadas = len(solicitacoes_mg_arquivadas)
print(f"Total de solicitações para o Estado de Minas Gerais com \nstatus 'Arquivado por decurso de prazo': {total_solicitacoes_mg_arquivadas}")

Total de solicitações para o Estado de Minas Gerais com 
status 'Arquivado por decurso de prazo': 1


In [45]:
# 3) Seu código aqui
df_bahia = consultar_dados_uf_tipo('BA', 'Município')
solicitacoes_bahia_deferidas = df_bahia[df_bahia['status'] == 'Deferido']
municipio_mais_solicitacoes_deferidas_bahia = solicitacoes_bahia_deferidas['interessado'].value_counts().idxmax()
print(f"Município da Bahia com mais solicitações deferidas: {municipio_mais_solicitacoes_deferidas_bahia}")

Município da Bahia com mais solicitações deferidas: Luís Eduardo Magalhães


In [46]:
# 4) Seu código aqui
df_bahia_estado = consultar_dados_uf_tipo('BA', 'Estado')
df_bahia_estado.to_csv('solicitacoes_bahia_estado.csv', index=False)